In [1]:
import requests
from bs4 import BeautifulSoup
import random

root_url = "https://liste-mots.com/dico-du-petit-bac/"
response = requests.get(root_url)

soup = BeautifulSoup(response.content, "html.parser")
a_tags = soup.find_all("a")

selected_urls = []

href = []
for a_tag in a_tags:
    href.append(a_tag.get("href"))

criteria = ["liste-des"]
for url in href:
    for criterion in criteria:
        if criterion in url:
            selected_urls.append(url)

print(selected_urls)

['https://liste-mots.com/dico-du-petit-bac/liste-des-animaux-de-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-animaux-de-a-a-z/liste-des-races-de-chats/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-animaux-de-a-a-z/liste-des-races-de-chiens-de-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-fruits-et-legumes/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-legumes-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-fruits-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-capitales-du-monde-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-pays-du-monde-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-villes-de-france/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-fleurs-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-fromages/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-couleurs-de-a-a-z/', 'https://liste-mots.com/dico-du-petit-bac/liste-des-pokemon-a-a-z/', 'https://lis

In [2]:
selected_urls_text_list = []
animals_dict = {}

category = ""

for urls in selected_urls:
    dico = {}
    selected_urls_text = requests.get(urls).text
    soup = BeautifulSoup(selected_urls_text, "html.parser")
    words = soup.find_all("p", class_="has-text-align-center")
    br_tags = soup.find_all("br")
    a_tags = soup.find_all("a")

    for br in br_tags:
        next_s = br.nextSibling
        if next_s and next_s.name is None:
            selected_urls_text_list.append(next_s.strip())

In [3]:
print(selected_urls_text_list)

['Abeille à miel', 'Abeille découpeuse', 'Abeille tueuse', 'Able de Heckel', 'Ablette', 'Acarien', 'Acarien rouge', 'Accenteur alpin', 'Lamproie', 'Lançon', 'Langouste', 'Langoustine', 'Langur', 'Lapin', 'Larinioides', 'Lemming', 'Achigan à grande bouche', 'Acouchi', 'Addax', 'Agneau', 'Agouti', 'Agrion élégant', 'Agrion jouvencelle', 'Aigle', 'Aigle royal', 'Aiglefin', 'Aigrette', 'Aiguillat', 'Aiguillat noir', 'Albatros', 'Alligator', 'Allocèbe', 'Alose', 'Alouate', 'Léopard', 'Léopard des neiges', 'Lépilémur', 'Lepture cordigère', 'Lérot', 'Lérotin', 'Leste fiancé', 'Léthocère d’Amérique', 'Lézard', 'Lézard à collier', 'Libellule', 'Liche', 'Lieu', 'Lieu jaune', 'Lieu noir', 'Lièvre', 'Lièvre de mer', 'Limace', 'Alpaga', 'Amazone impériale', 'Amie', 'Ammophile des sables', 'Amour blanc', 'Anaconda', 'Anax empereur', 'Anchois', 'Âne', 'Anémone de mer', 'Anguille', 'Angwantibo', 'Anhinga d’Amérique', 'Anoa', 'Anolis vert', 'Anomalure', 'Anophèle', 'Anoures', 'Antilope', 'Apelle', 'Apl

In [4]:
for category, letter, words in selected_urls_text_list:
    dico.setdefault(
        category,
        {},
    ).setdefault(
        letter, {}
    ).extend(words)
    # #letter #category #words

ValueError: too many values to unpack (expected 3)

In [ ]:
print(dico)

In [ ]:
# animals_dict = {}
# for word in selected_urls_text_list:
#     animals_dict.setdefault(word[0], []).append(word)
# print(animals_dict)

# a chaque fois que je vais sur une URL je créé un dico

In [ ]:
def find_word(letter):
    access_keys = animals_dict[letter]
    results = random.choice(access_keys)
    return "Animal : " + results


find_word("B")

KeyError: 'B'